In [1]:
import simulate
import baseline
import dataload
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#import pypots
from pypots.data import load_specific_dataset, mcar, masked_fill
from pypots.imputation import SAITS
from pypots.utils.metrics import cal_mae, cal_rmse, cal_mre

c:\Users\RUSHI\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# UCI HAR with baseline methods

In [4]:
dict = dataload.uci()
X = dict["X"]
y = dict["y"]

In [5]:
# MCAR
dict = simulate.simulate_nan(X, 0.1, "MCAR")
X_intact = dict['X_init']
X_missing = dict['X_incomp']
X_mask = dict['mask']

In [6]:
# Zero-filling imputation

X_zero_imputed = baseline.zero_filling_imputation(X_missing)

result = []
result.append([cal_mae(X_intact, X_zero_imputed), cal_rmse(X_intact, X_zero_imputed), cal_mre(X_intact, X_zero_imputed)])

In [7]:
# Mean imputation

X_mean_imputed = baseline.mean_imputation(X_missing)
result.append([cal_mae(X_intact, X_mean_imputed), cal_rmse(X_intact, X_mean_imputed), cal_mre(X_intact, X_mean_imputed)])

In [8]:
# Median imputation

X_median_imputed = baseline.median_imputation(X_missing)
result.append([cal_mae(X_intact, X_median_imputed), cal_rmse(X_intact, X_median_imputed), cal_mre(X_intact, X_median_imputed)])

In [9]:
result = pd.DataFrame(result, columns = ["mae", "rmse", "mre"], index=["zero-filling imputation", "mean imputation", "median imputation"])

In [10]:
result

,mae,rmse,mre
zero-filling imputation,0.011483,0.073733,9.364368e-09
mean imputation,0.011569,0.073462,9.302820e-09
median imputation,0.011354,0.074229,9.079908e-09


# PAMAPS2 with baseline methods

In [24]:
dict = dataload.pamaps2(window_size=40, drop_nan=True)
X = dict["X"]
y = dict["y"]

In [25]:
# MCAR

dict = simulate.simulate_nan(X, 0.1, "MCAR")
X_intact = dict['X_init']
X_missing = dict['X_incomp']
X_mask = dict['mask']

# X_intact, X_missing, missing_mask, X_mask = mcar(X, 0.1)
# X_missing = masked_fill(X_missing, 1 - missing_mask, np.nan)

In [26]:
# Zero-filling imputation

X_zero_imputed = baseline.zero_filling_imputation(X_missing)

result = []
result.append([cal_mae(X_intact, X_zero_imputed), cal_rmse(X_intact, X_zero_imputed, X_mask), cal_mre(X_intact, X_zero_imputed)])

In [27]:
# Mean imputation

X_mean_imputed = baseline.mean_imputation(X_missing)
result.append([cal_mae(X_intact, X_mean_imputed, X_mask), cal_rmse(X_intact, X_mean_imputed, X_mask), cal_mre(X_intact, X_mean_imputed, X_mask)])

In [28]:
# Median imputation

X_median_imputed = baseline.median_imputation(X_missing)
result.append([cal_mae(X_intact, X_median_imputed, X_mask), cal_rmse(X_intact, X_median_imputed, X_mask), cal_mre(X_intact, X_median_imputed, X_mask)])

In [29]:
result = pd.DataFrame(result, columns = ["mae", "rmse", "mre"], index=["zero-filling imputation", "mean imputation", "median imputation"])

In [30]:
result

,mae,rmse,mre
zero-filling imputation,4.296323,296.018871,3.757381e-10
mean imputation,1789.573271,2112.311897,9.799233e-01
median imputation,1789.551441,2112.324841,9.799231e-01
